In [2]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [3]:
AGENT_NAME = "ab000-hello-acp"

In [4]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.

from typing import cast
import uuid

from agentex.types import Task

TASK_ID = str(uuid.uuid4())[:8]

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="task/create",
    params={
        "name": f"{TASK_ID}-task",
        "params": {}
    }
)

task = cast(Task, rpc_response.result)
print(task)

Task(id='4a6ff681-8875-4a60-9e21-7308193dc327', created_at=datetime.datetime(2025, 7, 27, 1, 16, 2, 616, tzinfo=TzInfo(UTC)), name='110f7a13-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 7, 27, 1, 16, 2, 616, tzinfo=TzInfo(UTC)))


In [5]:
# Test non streaming response
from typing import cast
from agentex.types import Event

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="event/send",
    params={
        "content": {"type": "text", "author": "user", "content": "Hello what can you do?"},
        "task_id": task.id,
    }
)

event = cast(Event, rpc_response.result)
print(event)

Event(id='b235c5c6-b2f8-40b8-be60-a873c88f4d21', agent_id='93fa9758-abca-421e-a3ed-9f08a9881662', sequence_id=209, task_id='4a6ff681-8875-4a60-9e21-7308193dc327', content=TextContent(author='user', content='Hello what can you do?', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 7, 27, 1, 16, 2, 491008, tzinfo=TzInfo(UTC)))


In [6]:
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=5,
)

╭───────────────────────── USER [07/27/2025 01:16:02] ─────────────────────────╮
│ Hello what can you do?                                                       │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/27/2025 01:16:02] ─────────────────────────╮
│ Hello! I've received your message. I can't respond right now, but in future  │
│ tutorials we'll see how you can get me to intelligently respond to your      │
│ message.                                                                     │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 5 seconds - returning collected messages
